In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from pathlib import Path
import matplotlib

font = {"size": 12}
matplotlib.rc("font", **font)

In [ ]:
with Path("./evaluation_data.pkl").open("rb") as f:
    eval_data = pickle.load(f)
eval_data = np.array(eval_data)

(
    benchmarks,
    num_qubits,
    depth,
    num_all_gates,
    num_nonlocal_gates,
    program_communication,
    critical_depth,
    entanglement_ratio,
    parallelism,
    liveness,
) = [eval_data[:, i] for i in range(len(eval_data[0]))]
perc_multi = num_nonlocal_gates / num_all_gates

# Device Distribution

In [ ]:
overall_count_indep = 0
indep_qiskit = 0
indep_tket = 0

overall_count_nat = 0
nat_ibm = 0
nat_rigetti = 0
nat_oqc = 0
nat_ionq = 0

overall_count_mapped = 0
mapped_ibm_montreal_count = 0
mapped_ibm_washington_count = 0
mapped_oqc_lucy_count = 0
mapped_rigetti_count = 0
mapped_ionq_count = 0

qiskit_qubit_numbers = []
tket_qubit_numbers = []

for file in benchmarks:
    str_file = str(file)
    if "qiskit" in str_file:
        qiskit_qubit_numbers.append(int(str_file.split("_")[-1].split(".")[0]))
    elif "tket" in str_file:
        tket_qubit_numbers.append(int(str_file.split("_")[-1].split(".")[0]))

    if "indep" in str_file:
        overall_count_indep += 1
        if "indep_qiskit" in str_file:
            indep_qiskit += 1
        if "indep_tket" in str_file:
            indep_tket += 1

    if "nativegates" in str_file:
        overall_count_nat += 1
        if "nativegates_ibm" in str_file:
            nat_ibm += 1
        if "nativegates_rigetti" in str_file:
            nat_rigetti += 1
        if "nativegates_oqc" in str_file:
            nat_oqc += 1
        if "nativegates_ionq" in str_file:
            nat_ionq += 1
    elif "mapped" in str_file:
        overall_count_mapped += 1
        if "mapped_ibm_montreal" in str_file:
            mapped_ibm_montreal_count += 1
        if "mapped_ibm_washington" in str_file:
            mapped_ibm_washington_count += 1
        if "mapped_oqc_lucy" in str_file:
            mapped_oqc_lucy_count += 1
        if "mapped_rigetti" in str_file:
            mapped_rigetti_count += 1
        if "mapped_ionq" in str_file:
            mapped_ionq_count += 1

In [ ]:
perc_qiskit_indep = indep_qiskit / overall_count_indep
print(perc_qiskit_indep)
perc_tket_indep = indep_tket / overall_count_indep
print(perc_tket_indep)
print(perc_qiskit_indep + perc_tket_indep, overall_count_indep)

In [ ]:
perc_nat_ibm = nat_ibm / overall_count_nat
print(perc_nat_ibm)
perc_nat_rigetti = nat_rigetti / overall_count_nat
print(perc_nat_rigetti)
perc_nat_oqc = nat_oqc / overall_count_nat
print(perc_nat_oqc)
perc_nat_ionq = nat_ionq / overall_count_nat
print(perc_nat_ionq)
print(sum([perc_nat_ibm, perc_nat_rigetti, perc_nat_oqc, perc_nat_ionq]), overall_count_nat)

In [ ]:
perc_montreal = mapped_ibm_montreal_count / overall_count_mapped
print(perc_montreal)
perc_washington = mapped_ibm_washington_count / overall_count_mapped
print(perc_washington)
perc_oqc = mapped_oqc_lucy_count / overall_count_mapped
print(perc_oqc)
perc_rigetti = mapped_rigetti_count / overall_count_mapped
print(perc_rigetti)
perc_ionq = mapped_ionq_count / overall_count_mapped
print(perc_ionq)
print(sum([perc_montreal, perc_washington, perc_oqc, perc_rigetti, perc_ionq]), overall_count_mapped)

In [ ]:
data = [perc_washington, perc_rigetti, perc_montreal, perc_ionq, perc_oqc]
colors = ["#B7E6A5", "#7CCBA2", "#46AEA0", "#089099", "#00718B"]
labels = ["IBMQ Washington (127)", "Rigetti Aspen M2 (80)", "IBMQ Montreal (27)", "IonQ Harmony (11)", "OQC Lucy (8)"]
pies = plt.pie(data, labels=labels, autopct="%1.1f%%", colors=colors)
for pie in pies[0]:
    pie.set_alpha(0.6)
plt.savefig("results/pie.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Num Qubits

In [ ]:
import seaborn as sns

# colors = ['#7CCBA2', '#46AEA0', '#089099', "#00718B", "#045275", "#003147"]
# colors = ['#089099', '#003147']
plt.figure(figsize=(10, 5))
plot = sns.histplot(
    data=[qiskit_qubit_numbers, tket_qubit_numbers],
    bins=128,
    stat="density",
    alpha=0.3,
    kde=True,
    edgecolor="white",
    linewidth=0.5,
    palette=sns.color_palette("viridis", 2),
)
plt.legend(["TKET", "Qiskit"], reverse=True)
plt.ylabel("Relative Frequency")
plt.xlabel("Number of Qubits")
plt.savefig("results/qubit_dist.pdf", format="pdf", bbox_inches="tight")
plt.show()

# Further Characteristics

In [ ]:
datasets = [
    program_communication.astype(float),
    critical_depth.astype(float),
    entanglement_ratio.astype(float),
    liveness.astype(float),
    parallelism.astype(float),
    perc_multi.astype(float),
]
labels = [
    "Program Communication",
    "Critical Depth",
    "Entanglement Ratio",
    "Liveness",
    "Parallelism",
    "Percentage Multi-Qubit Gates",
]

In [ ]:
fig = plt.figure(figsize=(19, 4))
ax = fig.add_subplot(111)
colors = ["#7CCBA2", "#46AEA0", "#089099", "#00718B", "#045275", "#003147"]
pos = range(len(datasets))
plots = plt.violinplot(datasets, pos)
for pc, color in zip(plots["bodies"], colors):
    pc.set_facecolor(color)

ax.set_xticks(pos)
plt.xticks(rotation=0)
ax.set_xticklabels(labels)
plt.savefig("results/violins.pdf", format="pdf", bbox_inches="tight")